In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [2]:
# %fs   - for using in DataBricks, "spark" already runned
# ls /databricks-datasets

spark = SparkSession.builder\
    .master("local")\
    .appName("lambda")\
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/03/10 16:47:13 WARN util.Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/03/10 16:47:13 WARN util.Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [3]:
orders_df = spark.read\
   .option("header", True)\
   .option("inferSchema", True)\
   .csv("/datalake/bronze/dshop/orders/spark_stream/orders_2022-02-24.csv")

In [4]:
orders_schema = [i for i in orders_df.schema]

In [5]:
orders_schema

[StructField(order_id,IntegerType,true),
 StructField(product_id,IntegerType,true),
 StructField(client_id,IntegerType,true),
 StructField(store_id,IntegerType,true),
 StructField(quantity,IntegerType,true),
 StructField(order_date,StringType,true)]

In [6]:
csv_schema = StructType(orders_schema)

In [7]:
orders_data = spark.readStream\
    .schema(csv_schema)\
    .option("maxFilesPerTrigger", 1)\
    .csv("/datalake/bronze/dshop/orders/spark_stream/")

In [8]:
orders_data.writeStream\
    .format('memory')\
    .queryName('orders_stream')\
    .outputMode("append")\
    .start()

22/03/10 16:49:13 WARN streaming.StreamingQueryManager: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-c06af648-1bc2-4100-b8e4-6c610a025fdc. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.


22/03/10 16:50:08 ERROR executor.Executor: Exception in task 0.0 in stage 8.0 (TID 8)
java.lang.OutOfMemoryError: Java heap space
	at java.util.Arrays.copyOf(Arrays.java:3236)
	at java.io.ByteArrayOutputStream.grow(ByteArrayOutputStream.java:118)
	at java.io.ByteArrayOutputStream.ensureCapacity(ByteArrayOutputStream.java:93)
	at java.io.ByteArrayOutputStream.write(ByteArrayOutputStream.java:153)
	at org.apache.spark.util.ByteBufferOutputStream.write(ByteBufferOutputStream.scala:41)
	at java.io.ObjectOutputStream$BlockDataOutputStream.write(ObjectOutputStream.java:1853)
	at java.io.ObjectOutputStream.write(ObjectOutputStream.java:709)
	at org.apache.spark.util.Utils$.writeByteBuffer(Utils.scala:233)
	at org.apache.spark.scheduler.DirectTaskResult.$anonfun$writeExternal$1(TaskResult.scala:53)
	at org.apache.spark.scheduler.DirectTaskResult$$Lambda$2262/241322712.apply$mcV$sp(Unknown Source)
	at scala.runtime.java8.JFunction0$mcV$sp.apply(JFunction0$mcV$sp.java:23)
	at org.apache.spark.ut

In [9]:
orders_stream_df = spark.table("orders_stream").where(col('store_id') == 1)

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/home/user/.local/lib/python3.8/site-packages/py4j/java_gateway.py", line 1207, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/user/.local/lib/python3.8/site-packages/py4j/java_gateway.py", line 1033, in send_command
    response = connection.send_command(command)
  File "/home/user/.local/lib/python3.8/site-packages/py4j/java_gateway.py", line 1211, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while receiving
ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:40965)
Traceback (most recent call last):
  File "/home/user/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3251, in run_code


ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:40965)
Traceback (most recent call last):
  File "/home/user/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3251, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_24253/2671758561.py", line 1, in <module>
    orders_stream_df = spark.table("orders_stream").where(col('store_id') == 1)
  File "/home/user/.local/lib/python3.8/site-packages/pyspark/sql/session.py", line 741, in table
    return DataFrame(self._jsparkSession.table(tableName), self._wrapped)
  File "/home/user/.local/lib/python3.8/site-packages/py4j/java_gateway.py", line 1304, in __call__
    return_value = get_return_value(
  File "/home/user/.local/lib/python3.8/site-packages/pyspark/sql/utils.py", line 111, in deco
    return f(*a, **kw)
  File "/home/user/.local/lib/python3.8/site-packages/py4j/protocol.py", line 334, in get_return_value
    raise Py4JError(


ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:40965)
Traceback (most recent call last):
  File "/home/user/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3251, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_24253/2671758561.py", line 1, in <module>
    orders_stream_df = spark.table("orders_stream").where(col('store_id') == 1)
  File "/home/user/.local/lib/python3.8/site-packages/pyspark/sql/session.py", line 741, in table
    return DataFrame(self._jsparkSession.table(tableName), self._wrapped)
  File "/home/user/.local/lib/python3.8/site-packages/py4j/java_gateway.py", line 1304, in __call__
    return_value = get_return_value(
  File "/home/user/.local/lib/python3.8/site-packages/pyspark/sql/utils.py", line 111, in deco
    return f(*a, **kw)
  File "/home/user/.local/lib/python3.8/site-packages/py4j/protocol.py", line 334, in get_return_value
    raise Py4JError(


Py4JError: An error occurred while calling o23.table

In [10]:
orders_stream_df.head()

NameError: name 'orders_stream_df' is not defined

In [11]:
spark.sql(""" SELECT COUNT(*) FROM orders_stream WHERE store_id = 1""").show()
# %sql
# select count(*) from orders_stream where store_id = 1    # DataBricks selecting

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:40965)
Traceback (most recent call last):
  File "/home/user/.local/lib/python3.8/site-packages/py4j/java_gateway.py", line 977, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/user/.local/lib/python3.8/site-packages/py4j/java_gateway.py", line 1115, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 111] Connection refused


Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:40965)

In [ ]:
orders_stream_df = orders_stream_df

In [ ]:
client_df = spark.read\
    .option("header", True)\
    .option("inferSchema", True)\
    .parquet("/datalake/silver/dshop/clients/")

In [ ]:
client_df.show()